# CE263N Final Project
Minho Kim (PhD, Landscape Arch. & Env. Planning)
Keats Hua (Masters, Civil & Env. Engineering)

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

Installing geemap ...
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
  Created wheel for ee-extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236772 sha256=42cb50ebe55215bfd6b78a92565ef3f64fb1b5b56a5f8397d8adac2ba1425680
  Stored in directory: /Users/MacBookPro/Library/Caches/pip/wheels/66/66/06/98d6dee3f612d84d2b487fc73ca2c2479279dc2714faddcc10
  Created wheel for sankee: filename=sankee-0.2.1-py3-none-any.whl size=29337 sha256=23e213aa176cf92b4bf5a9094f3de8fdecfd877672ad9a0e1d3a142304180eb7
  Stored in directory: /Users/MacBookPro/Library/Caches/pip/wheels/62/05/82/93dd0850c076353f7360e29d657827713639e92f7076052564
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492036 sha256=1d2f092af54539708b4006bfdd24a629a

In [2]:
# ee.Authenticate()

In [3]:
import ee
ee.Initialize()

import geemap
import pandas as pd
import os
import rasterio
import matplotlib.pyplot as plt

In [4]:
def subsetNAIP(img_col, startTime, endTime, fc):
    img = img_col.filterDate(startTime, endTime).filterBounds(fc).mosaic().clip(fc)
    return img

def calNDVI(image):
    return image.normalizedDifference(['R', 'N'])

def rasterToVector(img, fc):
    vec = img.reduceToVectors(geometry=fc, eightConnected=True, maxPixels=59568116121, crs=img.projection(), scale=1)
    return vec   

## Main Code
- Import main functions
- Import collections and data --> Filter Date + Bounds
- Preprocess each dataset
- Export datasets to drive

#### Datasets
- NAIP --> NDVI, NDWI, EVI, GCI, FVC
- DEM (3DEP) --> Elevation (as-is), slope, aspect, hillshade
- LULC (ESA)
- Wind (GRIDMET)
- Fuel Model (LANDFIRE)

| Data Source | Spatial Resolution | Temporal Resolution | Extractable Features |
| :-: | :-: | :-: | :-: | 
| NAIP | < 1m | Yearly | NDVI, NDWI, EVI, GCI, FVC |
| DEM (3DEP) | ~10m | Variable | Elevation, Slope, Aspect, Hillshade |
| LULC (ESA) | 10m | Yearly | Land use, Land cover |
| Wind (GRIDMET) | 4.6km | ? | Wind Direction, Wind Velocity | 
| Fuel Model (LANDFIRE) | 30m | ? | Fuel Classes |

### Input Parameters

In [5]:
# Set input parameters and file directory
os.chdir('/Users/minho/Desktop/CE263/Project')

years = [2018] # For NAIP
img_scale = 10 # For output's spatial resolution

### Read AOI csv

In [6]:
def aoi_in(long1, lat1, long2, lat2):
    return ee.FeatureCollection(ee.Geometry.Polygon([[[long1, lat1],[long1, lat2],[long2, lat2],[long2, lat1]]]))

'''
Example 

aoi = ee.FeatureCollection(ee.Geometry.Polygon(
        [[[-122.2363652412898, 37.9181758764888],
          [-122.2363652412898, 37.83091254476254],
          [-122.11894885945387, 37.83091254476254],
          [-122.11894885945387, 37.9181758764888]]]
        )
      )
      
'''

# Input UL and LR long/lat values for each AOI polygon (as ee.FeatureCollection) into list
aoi_csv = pd.read_csv('aoi_list.csv')

Map = geemap.Map(center=[40,-100], zoom=4)

aois = []
years, months, days = [],[],[]

for i in range(len(aoi_csv)):
    aois.append(aoi_in(aoi_csv['long1'][i], aoi_csv['lat1'][i], aoi_csv['long2'][i],aoi_csv['lat2'][i]))
    
    years.append('20'+aoi_csv['date'][i].split('/')[-1])
    months.append(aoi_csv['date'][i].split('/')[-3].zfill(2))
    days.append(aoi_csv['date'][i].split('/')[-2].zfill(2))
    
aoi_list = pd.DataFrame()
aoi_list['year'] = years
aoi_list['month'] = months
aoi_list['day'] = days
aoi_list['aoi'] = aois

In [7]:
# Number of AOI (Polygons) in list

if type(aoi_list) == ee.featurecollection.FeatureCollection:
    count = aoi.size().getInfo()    
else:
    count = len(aoi_list)
    
print('Number of AOI polygons: ', count)

Number of AOI polygons:  51


In [8]:
# # Convert all AOI polygons to "shapefiles" and save to disc
# for i in range(len(aoi_list)):
    
#     task = ee.batch.Export.table.toDrive(**{
#       'collection': aois[i],
#       'description':'fire_aoi'+str(i),
#       'folder':'fires',
#       'fileFormat': 'SHP'
#     })
    
#     task.start()

## Input Datasets
- DEM
- LULC
- GRIDMET
- Satellite Images (Sentinel-2, NAIP)
- LANDFIRE Vegetation height, vegetation condition (For fuel model)

### DEM

In [9]:
# DEM
dem = ee.Image('USGS/3DEP/10m')

elevation = dem.select('elevation')
slope = ee.Terrain.slope(elevation)

# x, y = np.gradient(dem)
# aspect = np.arctan2(-x, y)

### LULC

In [10]:
# LULC
lulc = ee.ImageCollection("ESA/WorldCover/v100").first();

### GRIDMET (Weather)

In [11]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

# Variables (maxtemp, mintemp, minhumidity, maxhumidity, winddir, windvel, precip)
maxtemp = gridmet.select('tmmx')
mintemp = gridmet.select('tmmn')

minhumidity = gridmet.select('rmin')
maxhumidity = gridmet.select('rmax')

winddir = gridmet.select('vs') # Degrees clockwise from North
windvel = gridmet.select('th') 

precip = gridmet.select('pr')

# maxtempvis = {
#   min: 290.0,
#   max: 314.0,
#   'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c'],
# };
# Map.setCenter(-115.356, 38.686, 15)
# Map.addLayer(maxtemp, maxtempvis, 'Max Temp')


### LANDFIRE: Vegetation Height + Condition (2014)

In [13]:
evh = ee.ImageCollection('LANDFIRE/Vegetation/EVH/v1_4_0').select(['EVH'])
vcc = ee.ImageCollection('LANDFIRE/Fire/VCC/v1_4_0').first

In [16]:
def perc_cov(img, aoi_ext, band_name):

    # count pixels in aoi image
    totPixels = ee.Number(ee.Image(1).reduceRegion(
        reducer= ee.Reducer.count(),
        geometry= aoi_ext,
        scale= 10,
        maxPixels= 999999999,
      ).values().get(0))

    # Count the non zero/null pixels in the image within the aoi
    actPixels = ee.Number(img.select(band_name).reduceRegion(
        reducer= ee.Reducer.count(),
        scale= 10,
        geometry= aoi_ext, 
        maxPixels= 999999999,
      ).values().get(0))

    # calculate the perc of cover
    pcPix = actPixels.divide(totPixels).multiply(100)
#     print('AOI Coverage : ', actPixels.getInfo(), 'pixels out of ', totPixels.getInfo())
#     print('Percentage Cover {:.3f} %'.format(pcPix.getInfo()))
    
    return pcPix

### Filter S2

In [17]:
def calc_stats(img, roi):
    imgPoly = ee.Algorithms.GeometryConstructors.Polygon(
              ee.Geometry( img.get('system:footprint') ).coordinates()
              )

    intersection = roi.intersection(imgPoly, ee.ErrorMargin(0.5))
    coveragePercent = intersection.area().divide(roi.area(0.001)).multiply(100)

    img = img.set('ROI_COVERAGE_PERCENT', coveragePercent)
    return coveragePercent

In [18]:
# Sentinel-2
s2_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
s2_10 =  ['B2', 'B3', 'B4']
# s2vis = {min: 0.0, max: 0.5, 'bands':['B4', 'B3', 'B2']}

s2vis = {
  min: 0.0,
  max: 2000,
  'bands': ['B4', 'B3', 'B2'],
}


def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


def s2_ee(year, month, day, filterMo, aoi):

    startMo = str((int(month) - filterMo)).zfill(2)

    # Set date of acquisition
    if month == '01':
        startTime = ee.Date(str(int(year)-1) + '-' + str('12') + '-' + str(day))

    elif startMo == '04' and day == '31' or startMo == '06' and day == '31' or startMo == '09' and day == '31' or startMo == '11' and day == '31':
        startTime = ee.Date(str(year) + '-' + str(startMo) + '-' + str('30'))
    else: 
        startTime = ee.Date(str(year) + '-' + str(startMo) + '-' + str(day))
    
    endTime = ee.Date(str(year) + '-' + str(month) + '-' + str(day))
    print(startTime.format('Y-MM-dd').getInfo())
    
    # Set map center and visualize
    polys = aoi.geometry()
    centroid = polys.centroid()
    lng, lat = centroid.getInfo()['coordinates']
    
    s2_list = ee.ImageCollection('COPERNICUS/S2') \
                    .filterDate(startTime, endTime) \
                    .filterBounds(aoi) \
                    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",2)) \
                    .sort('CLOUDY_PIXEL_PERCENTAGE') \
#                     .map(maskS2clouds)
    print(" Number of images: ", s2_list.size().getInfo())

    
    # Filter through by AOI coverage percentage (%)
    filtered_list = pd.DataFrame(columns = ['date', 'image'])
    good_list, date_list, img_id = [],[],[]

    listOfImages = s2_list.toList(s2_list.size())

    for i in range(len(listOfImages.getInfo())):   

        pcpix = calc_stats(ee.Image(listOfImages.get(i)), aoi.geometry()) # Set input AOI as "geometry"
        pcpix_thresh = pcpix.getInfo()

        if pcpix_thresh > 99: # AOI coverage threshold set to 90%
            # rank_list.append(i)
            good_list.append(ee.Image(listOfImages.get(i)))
            img_id.append(ee.Image(listOfImages.get(i)).get('system:index').getInfo())
            date_list.append(ee.Date(ee.Image(listOfImages.get(i)).get('system:time_start')).format('Y-MM-dd').getInfo())
            print("Date of image : ", ee.Date(ee.Image(listOfImages.get(i)).get('system:time_start')).format('Y-MM-dd').getInfo())        
            
#             filtered_list['rank'] = rank_list
            filtered_list['image'] = good_list
            filtered_list['date'] = date_list
            filtered_list['id'] = img_id
            
            break

    if len(filtered_list) > 0:
        s2_filtered = filtered_list['image'][0].select(s2_10).clip(polys)
    else:
        s2_filtered = 0
        
#     filtered_list['rank'] = rank_list
#     filtered_list['image'] = good_list
#     filtered_list['date'] = date_list
#     filtered_list['id'] = img_id

#     print(filtered_list)
    
#     if len(filtered_list) > 0:
#         s2_filtered = filtered_list['image'][0].select(s2_10).clip(polys)
        
#     Map.addLayer(s2_filtered, {min:0, max:1000, 'bands':['B4','B3','B2']}, 'filtered'+str(i))
    
    return s2_filtered,filtered_list
    
#     # Mask clouds with QA60 layer
#     s2_list_qa = s2_list.map(maskS2clouds)  
#     s2filt = ee.Image(s2_list.first()).select(s2_10)
#     s2filtqa = ee.Image(s2_list_qa.first()).select(s2_10)
#     date = ee.Date(s2filt.get('system:time_start'))
#     print("Formatted date", date.format('Y-MM-dd').getInfo())

#     # Set filename for S2 imagery
    
#     filename = 'S2' + str(year) + '_' + str(i) + '.tif'
    
#     s2clipped = s2filt.clip(polys)
#     s2ndvi = s2clipped.normalizedDifference(['B8','B4'])

# #     Map.setCenter(lng, lat, 14)
# #     Map.addLayer(s2clipped, {}, 'S2')
# #     Map.addLayer(s2ndvi, {}, 'S2_ndvi')
# #     Map
#     return s2clipped, s2filtqa.clip(polys), filtered_list


In [19]:
ult_list = pd.DataFrame()

In [20]:
for i in range(len(aoi_list)):
    
    print('AOI # {:.0f}'.format(i))
    year = aoi_list['year'][i]
    month = aoi_list['month'][i]
    day = aoi_list['day'][i]

    img, filtered_list = s2_ee(year, month, day, 3, aoi_list['aoi'][i])

    filtered_list['aoi'] = i
    ult_list = ult_list.append(filtered_list)

AOI # 0
2021-04-04
 Number of images:  88
Date of image :  2021-04-06
AOI # 1
2021-04-03
 Number of images:  47
Date of image :  2021-06-28
AOI # 2
2021-05-15
 Number of images:  147
AOI # 3
2021-04-14
 Number of images:  256
AOI # 4
2021-04-01
 Number of images:  92
Date of image :  2021-04-01
AOI # 5
2021-05-29
 Number of images:  8
Date of image :  2021-06-16
AOI # 6
2021-03-25
 Number of images:  33
AOI # 7
2021-04-30
 Number of images:  32
AOI # 8
2021-03-30
 Number of images:  19
Date of image :  2021-04-09
AOI # 9
2021-03-28
 Number of images:  17
Date of image :  2021-03-30
AOI # 10
2020-05-30
 Number of images:  14
Date of image :  2020-06-30
AOI # 11
2020-04-30
 Number of images:  39
Date of image :  2020-07-17
AOI # 12
2020-05-17
 Number of images:  31
AOI # 13
2020-05-17
 Number of images:  79
AOI # 14
2020-06-06
 Number of images:  37
AOI # 15
2020-06-29
 Number of images:  46
AOI # 16
2020-06-09
 Number of images:  9
Date of image :  2020-08-10
AOI # 17
2020-06-18
 Number

In [22]:
df = ult_list.reset_index(drop=True)

### Download images (Task)

In [ ]:
for i in range(2):
    print("AOI #", i)
    a=aoi_list['aoi'][i].geometry().getInfo()['coordinates']
#     print("UL: ", a[0][0], " and LR: ", a[0][3])
    print(aoi_list['aoi'][df['aoi'][i]].geometry().getInfo()['coordinates'])

In [ ]:
for i in range(len(df)):

    a=aoi_list['aoi'][i].geometry().getInfo()['coordinates']
    print("UL: ", a[0][0], " and LR: ", a[0][3])

    task1 = ee.batch.Export.image.toDrive(image=df['image'][i].select(['B2','B3','B4','B8']).clipToCollection(aoi_list['aoi'][df['aoi'][i]]),
                                          description='s2_aoi_ll'+str(df['aoi'][i]),
                                          folder='fires', 
#                                           crs=df['image'][i].select('B2').projection().getInfo()['crs'],
                                          crs= 'EPSG:4326',
                                          maxPixels=1e13, 
                                          scale=10, 
                                          fileFormat='GeoTIFF',
#                                           region=aoi_list['aoi'][df['aoi'][i]].geometry().bounds())
                                          region=aoi_list['aoi'][df['aoi'][i]].geometry().getInfo()['coordinates'])


    task1.start()        

In [ ]:
task1.status()

In [ ]:
for i in range(len(df)):

    task2 = ee.batch.Export.image.toDrive(image=df['image'][i].select(['B5','B6','B7','B11','B12']).clipToCollection(aoi_list['aoi'][df['aoi'][i]]),
                                          description='s2_20m_aoi'+str(df['aoi'][i]),
                                          folder='fires', 
                                          crs=df['image'][i].select('B2').projection().getInfo()['crs'],
                                          maxPixels=1e13, 
                                          scale=20, 
                                          fileFormat='GeoTIFF',
#                                           region=aoi_list['aoi'][df['aoi'][i]].geometry().bounds())
                                          
    task2.start()        

In [ ]:
task2.status()

## Visualize map layers

In [ ]:
for i in range(len(df)):
    Map.addLayer(df['image'][11], {min:0, max:1000, 'bands':['B4','B3','B2']}, 'filtered'+str(i))

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map.setCenter(aoi_list['aoi'][11].geometry().centroid().getInfo()['coordinates'], 14)
Map.addLayer(df['image'][11].clip(aoi_list['aoi'][11]), {min:0, max:6000, 'bands':['B4','B3','B2']}, 'test_img')

In [ ]:
Map.addLayer(filtered_list['image'][0].normalizedDifference(['B8','B4']).clip(aoi_list['aoi'][0].geometry()), {}, 'ndvi_t')

### NAIP imagery + NDVI (Visualization + Download)

In [ ]:
# NAIP

# truevis = {'bands': ['R', 'G', 'B'],'min':0}
# ndviViz = {min: 0, max: 1, 'palette':['ffffff', '009900']}

# for year in years:

    # Set date of acquisition
#     startTime = ee.Date(str(year) + '-01-01')
#     endTime = ee.Date(str(year) + '-12-31')

    # Set filename for NAIP imagery
#     filename = 'NAIP_' + str(year) + '_' + str(i) + '.tif'

    # Set map center and visualize
#     polys = aoi_list['aoi'][i].geometry()
#     centroid = polys.centroid()
#     lng, lat = centroid.getInfo()['coordinates']
#     print("lng = {}, lat = {}".format(lng, lat))

    # NAIP filter
#     naip_img = ee.ImageCollection('USDA/NAIP/DOQQ').filterDate(startTime, endTime).filterBounds(aois[i]).mosaic().clip(aois[i])
#     ndvi = naip_img.normalizedDifference(['N', 'R'])

for i in range(len(aoi_list)):

    # Variables (maxtemp, mintemp, minhumidity, maxhumidity, winddir, windvel, precip)
    dl_img = [evh, vcc]
    dl_dict = {
            evh: 'evh_',
            vcc: 'vcc_',
#             maxtemp: 'maxtemp_', 
#             mintemp: 'mintemp_',
#             minhumidity: 'minhumidity_', 
#             maxhumidity: 'maxhumidity_',
#             winddir: 'winddir_',
#             windvel: 'windvel_', 
#             precip: 'precip_'
            }

    # Download images and save to Google Drive
    for img in dl_img:

        print("Running : ", dl_dict[img], "for AOI" , i)

        task2 = ee.batch.Export.image.toDrive(image=ee.Image(img).clip(aoi_list['aoi'][i].geometry()),
                                              description=dl_dict[img]+'aoi'+str(i),
                                              folder='fires', 
                                              crs='EPSG:4326',
                                              maxPixels=1e13, 
                                              scale=10, 
                                              fileFormat='GeoTIFF',
                                              region=aoi_list['aoi'][i].geometry())

        task2.start()        

#     Map.setCenter(lng, lat, 14)
#     Map.addLayer(naip_img, truevis, 'truevis')
#     Map.addLayer(ndvi, ndviViz, 'ndvi')
#     Map.addLayer(slope.clip(aoi_list[i]), {min: 0, max: 60}, 'slope')
#     Map.addLayer(dem.clip(aoi_list[i]), {}, 'dem')
#     Map.addLayer(lulc.clip(aoi_list[i]), {}, 'lulc')

#         Map.addLayer(maxtemp.mosaic().clip(aoi_list[i]), {}, 'maxtemp')
#         Map.addLayer(mintemp.mosaic().clip(aoi_list[i]), {}, 'mintemp')
#         Map.addLayer(minhumidity.mosaic().clip(aoi_list[i]), {}, 'minhumidity')
#         Map.addLayer(maxhumidity.mosaic().clip(aoi_list[i]), {}, 'maxhumidity')
#         Map.addLayer(winddir.mosaic().clip(aoi_list[i]), {}, 'winddir')
#         Map.addLayer(windvel.mosaic().clip(aoi_list[i]), {}, 'windvel')
#         Map.addLayer(precip.mosaic().clip(aoi_list[i]), {}, 'precip')        



#     Map.addLayer(polys)



In [ ]:
task2.status()